<a href="https://colab.research.google.com/github/VictorVazquezRey/03MAIR--Algoritmos-de-Optimizacion--2020/blob/master/Victor_Vazquez_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Actividad guiada 3 de Algoritmos de Optimización**
*Victor Vazquez Rey*

[https://colab.research.google.com/drive/13Uv4Q6qaV0lO089NDyq5UbBjd4WwTtCZ?usp=sharing](https://colab.research.google.com/drive/13Uv4Q6qaV0lO089NDyq5UbBjd4WwTtCZ?usp=sharing)

[https://github.com/VictorVazquezRey/03MAIR--Algoritmos-de-Optimizacion--2020](https://github.com/VictorVazquezRey/03MAIR--Algoritmos-de-Optimizacion--2020)

#### **Preparación**
Vamos a instalar los paquetes para descargar el archivo tsp que se hace uso en esta práctica y que representa el problema del vendedor viajero en 42 ciudades de Suiza.

In [ ]:
!pip install request
!pip install tsplib95

  Created wheel for request: filename=request-0.0.0-cp36-none-any.whl size=1326 sha256=8a3a149b64e431bbb162dd8eef7c1eb8b0520e95ee042ef7bdda9b9a6755bb94
  Stored in directory: /root/.cache/pip/wheels/d0/dd/4d/b2d488171e9e1129106e8134af9f2655f62dd17edd9d9a6b5f
  Created wheel for get: filename=get-0.0.0-cp36-none-any.whl size=1324 sha256=b97e486df9e8c820c4a393044601fefb3c313d857828179443bf9adfd38f272b
  Stored in directory: /root/.cache/pip/wheels/0e/d1/1d/52b42199057663cc6cd7f012086fee189e1c0bd851fd7b6fd3
  Created wheel for post: filename=post-0.0.0-cp36-none-any.whl size=1261 sha256=380e579ab4047ca59c48ee09afbd3aeb3499fac1757e81cc719c0e2dab2f04b2
  Stored in directory: /root/.cache/pip/wheels/44/74/40/aac3b48aa3d964c28a29cc998adfaf69ed90aad30ee738ba9f
  Created wheel for query-string: filename=query_string-0.0.0-cp36-none-any.whl size=1510 sha256=053a0870fcd16e3c81f4e3b52096391cbde72721135ef6b354bc56d56b4a9ac7
  Stored in directory: /root/.cache/pip/wheels/b5/ad/d6/60f8dc1772dd7abe21c

Hemos implementado una clase auxiliar cuya responsabilidad es mantener las funciones comunes a todas las estrategias de resolución del problema y de contener los datos descargados.

A esa clase la hemos denominado TSP por Traveling Salesman Problem e incluye funcionalidades como el cálculo de una solución aleatoria, o el coste de una solución o el de una arista.

La funcionalidad es la que se espera según el nombre de las funciones, aunque destacaría el uso de shuffle que ya baraja aleatoriamente una lista dada y que nos viene perfecto al crear una nueva solución (el slice se hace a partir de uno porque en la resolución de nuestro problema las soluciones siempre empiezan y acaban en 0.

Se usa el esquema map/reduce para el cáculo del coste y el segundo sumando es el coste de volver al inicio.

In [ ]:
from requests.sessions import Session
import tsplib95
import numpy as np
from functools import reduce
from operator import add
from random import sample, random, choice

class TSP:
    def __init__(self, url: str):
        with Session() as s:
            response = s.get(url)
            content = response.content.decode('utf-8')

        self.problem = tsplib95.parse(content)
        self.nodes = list(self.problem.get_nodes())
        self.edges = list(self.problem.get_edges())
        self.dimension = len(self.nodes)

    def create_solution(self) -> list:
        solution = np.array(self.nodes)
        np.random.shuffle(solution[1:])
        return solution

    def weight(self, a: int, b: int) -> int:
        return self.problem.get_weight(a, b)

    def distance(self, solution: list) -> int:
        return reduce(add, map(self.weight, solution[:-1], solution[1:])) + \
               self.weight(solution[self.dimension - 1], solution[0])

    def swap(self, solution: 'nparray', pos1: int, pos2: int):
        solution[pos1], solution[pos2] = solution[pos2], solution[pos1]

    def find_neighbor(self, solution: 'nparray'):
        i, j = sample(range(1, self.dimension), 2)
        neighbor = np.copy(solution)
        self.swap(neighbor, i, j)
        distance_neighbor = self.distance(neighbor)

        return neighbor, distance_neighbor

Con estas líneas de código mantemos un objeto que el resto de estrategias van a poder usar.

In [ ]:
URL = 'http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp'
tsp = TSP(URL)

#### **Algoritmo de búsqueda aleatoria**
Es la primera aproximación. Se hacen una serie configurable de iteraciones con generación automática de soluciones y se elige la mejor de las generadas.

El código aplica un esquema de recorrido en función del parámetro de iteraciones para quedarse con el mínimo de las generadas.

In [ ]:
class RandomSearch:
    def __init__(self, tsp:TSP, iterations: int = 100):
        self.tsp = tsp
        self.iterations = iterations

    def random_search(self, iterations: int = 100) -> int:

        best_solution = self.tsp.create_solution()
        distance_best_solution = self.tsp.distance(best_solution)

        for it in range(iterations):
            actual_solution = self.tsp.create_solution()
            distance_actual_solution = self.tsp.distance(actual_solution)

            if distance_actual_solution < distance_best_solution:
                best_solution = actual_solution
                distance_best_solution = distance_actual_solution

        return best_solution, distance_best_solution

##### **Solución para la búsqueda aleatoria**

Definida la clase el cálculo de la solución se ejecuta con estas tres líneas. A partir de este punto haremos clases similares para el resto de los algoritmos.

In [ ]:
rs = RandomSearch(tsp)

solution, distance = rs.random_search()
print("La mejor solución es {} con una distancia de {}".format(solution, distance))

La mejor solución es [ 0 30  6 12 23 19  2 33 28 37  1 36 29 32 22  9 40 39 18 25 10 31 14 16
  5 38  3 26  4 34  8 11 13  7 15 27 20 21 24 35 41 17] con una distancia de 4270


#### **Algoritmo de búsqueda Local**
Este algoritmo intensifica la solución buscando el mejor vecino para una solución inicial aleatoria hasta que ya no puede mejorar esa solución.

Entre todas las posibilidades para generar una búsqueda local esta se denomina de best-improvement por elejir el mejor vecino posible considerando vecino a cualquier intercambio de dos posiciones en la solución original.

Para ello se hace uso de la función swap del objeto tsp de la clase TSP que efectúa el intercambio.

También resalto el uso de combinations que devuelve todas las posibles combinaciones tomadas de k en k (de dos en dos en este caso por el segundo parámetro)

In [ ]:
import numpy as np
from itertools import combinations


class LocalSearch:
    def __init__(self, tsp:TSP):
        self.tsp = tsp

    # best-improvement
    def find_best_neighbor(self, solution: 'nparray'):

        best_solution = solution
        distance_best_solution = self.tsp.distance(solution)

        for i, j in combinations(np.array(self.tsp.nodes[1:]), 2):
            neighbor = np.copy(solution)
            self.tsp.swap(neighbor, i, j)
            distance_neighbor = self.tsp.distance(neighbor)

            if distance_neighbor < distance_best_solution:
                distance_best_solution = distance_neighbor
                best_solution = neighbor

        return best_solution, distance_best_solution

    def local_search(self):

        solution = self.tsp.create_solution()
        best_solution = solution
        distance_best_solution = self.tsp.distance(best_solution)

        while True:
            neighbor, distance_neighbor = self.find_best_neighbor(solution)
            if distance_neighbor < distance_best_solution:
                best_solution = neighbor
                distance_best_solution = distance_neighbor
            else:
                return best_solution, distance_best_solution

            solution = neighbor

##### **Solución para la búsqueda Local**

In [ ]:
rs = LocalSearch(tsp)

solution, distance = rs.local_search()
print("La mejor solución es {} con una distancia de {}".format(solution, distance))

La mejor solución es [ 0  1  6  5  4 32 20 33 34 24 40 21 39 22 38 30 28 31 35 36 17 29  9 23
 41 25 10  8  2 27  3  7 37 15 16 14 19 13 11 12 18 26] con una distancia de 1780


#### **Algoritmo de recocido simulado**
Se basa en el proceso físico de templado.

Va a permitir en determinados casos escoger soluciones peores y así permitir salir de la localidad en función de una abtracción de la temperatura y una función de probabilidad.

Esta implementación admite diferentes formas de descenso de la temperatura y para ello se dispone de un enumerado que se puede pasar por parámetro tanto en la creación de la clase como en la llamada al método principal.

In [ ]:
from enum import Enum
from math import exp, log
from random import random


class Descend_Type(Enum):
    EXPONENTIAL = 1
    BOLTZMAN = 2
    CAUCHY = 3


class SimulatedAnnealing:
    def __init__(self, tsp: TSP, temperature: float = 1000,
                 descend_type=Descend_Type.EXPONENTIAL, alpha: float = .999):
        self.descend_type = descend_type
        self.tsp = tsp
        self.temperature_0 = temperature
        self.temperature_k = temperature
        self.k = 1
        self.alpha = alpha

    def lower_the_temperature(self):
        if self.descend_type == Descend_Type.EXPONENTIAL:
            self.temperature_k *= self.alpha
        elif self.descend_type == Descend_Type.BOLTZMAN:
            self.temperature_k = self.temperature_0 / (1 + log(self.k))
            self.k += 1
        elif self.descend_type == Descend_Type.CAUCHY:
            self.temperature_k = self.temperature_0 / (1 + self.k)
            self.k += 1
        else:
            raise ValueError('Descend_Type unkown')

    # If cost of neighbour is lower is always accepted, otherwise it depends
    # on temperature and the difference of the costs.
    # We introduce some ramdomness too. 
    def is_accepted(self, d_n: int, d_s: int):
        if d_n >= d_s:
            return random() <= exp(-1 * (d_n - d_s) / self.temperature_k)
        return True

    def simulated_annealing(self, temperature: float = 1000, delta: float = .0001,
                            descend_type=Descend_Type.EXPONENTIAL, alpha: float = .999):
        self.temperature_0 = temperature
        self.temperature_k = temperature
        self.descend_type = descend_type
        self.alpha = alpha
        solution = self.tsp.create_solution()
        distance_solution = self.tsp.distance(solution)
        best_solution = solution
        distance_best_solution = distance_solution

        while self.temperature_k > delta:
            neighbor, distance_neighbor = self.tsp.find_neighbor(solution)
            if distance_neighbor < distance_best_solution:
                best_solution = neighbor
                distance_best_solution = distance_neighbor

            if self.is_accepted(distance_neighbor, distance_solution):
                solution = neighbor
                distance_solution = distance_neighbor

            self.lower_the_temperature()

        return best_solution, distance_best_solution

##### **Solución para la búsqueda por recocido simulado**

In [ ]:
rs = SimulatedAnnealing(tsp)

solution, distance = rs.simulated_annealing()
print("La mejor solución es {} con una distancia de {}".format(solution, distance))

La mejor solución es [ 0 17 35 36 37  7  6 26 18 12 11 25 10  4  1 31 20 33 24 40 21 39  9  8
 13 19 16 15 14  5 41 23 22 38 34 32  3 27  2 29 30 28] con una distancia de 1975


#### **Algoritmo de colonia de hormigas**

Se han implementado las mejoras para la elección del nodo siguiente y el descenso de la temperatura depende de un parámetro rho configurable o tasa de evaporación y el incremento de un parámetro q o multiplicador de la inversa del coste, así a menos coste mayor cantidad añadida de feromona.

Adicionalmente se han creado generaciones de hormigas, mejorando la aproximación inicial con una única generación a generation_number generaciones.

Esta implementación tendrá por tanto varios hiperparámetros, los más relevantes además de los ya descritos son $\alpha$ y $\beta$ que permiten modificar la probabilidad de elección de nodo a expandir de una hormiga para poner más peso en la feromona o en la llamada deseabilidad que no es más que la inversa del coste.

Destacaría el uso de la función np.random.choice que devuelve un array de elecciones no uniformes a partir de un array valores y uno de probabilidades de elección sobre el primer array con la misma dimensión. De esta forma se ha calculado el array de probabilidades según la formula:

$p^k_{ij}(t) = \frac{[\tau_{ij}(t)]^\alpha[\nu_{ij}]^\beta}{\sum_{l\in J^k_i} [\tau_{il}(t)]^\alpha[\nu_{il}]^\beta}$, si $j \in J^k_i$

$p^k_{ij}(t) = 0$, si $j \notin J^k_i$

y luego de las aristas alcanzables se elige con choice el nodo siguiente.

In [ ]:
from functools import reduce
from operator import add
import numpy as np
from math import inf

class AntColony:
    def __init__(self, tsp:TSP):
        self.tsp = tsp
        self.trails = {}
        self.ants = []
        self.ants_distances = []
        self.alpha = .1
        self.beta = .1
        self.rho = .1
        self.generations_number = 10
        self.number_of_ants = 100
        self.initial_theta = .1
        self.Q =1000

    def init_trails(self):
        self.trails = {k: 1 for k in self.tsp.edges}

    def init_ants(self):
        self.ants = [[0] for _ in range(self.number_of_ants)]
        self.ants_distances = [inf for _ in range(self.number_of_ants)]

    def desiderability(self, edge : tuple) -> float:
        return 1/self.tsp.weight(edge[0], edge[1])

    def choose_next_node(self, ant: int):
        # this calculates reachable edges from last position visited by ant
        last_node = self.ants[ant][-1]
        left_nodes = list(set(self.tsp.nodes)-set(self.ants[ant]))
        reachable_edges = [edge for edge in tsp.edges if edge[0] == last_node and edge[1] in left_nodes]
        # calculate denominator of probability term
        sum_factors = reduce(add, [(self.trails[edge]**self.alpha) * (self.desiderability(edge)**self.beta) for edge in reachable_edges])
        # list of probabilities for each edge to be selected
        probabilities = [(self.trails[edge]**self.alpha) * (self.desiderability(edge)**self.beta) / sum_factors for edge in reachable_edges]
        # choice a position with calculated probabilities return a list of positions (only one because the second parameter)
        to_return = np.random.choice(range(len(probabilities)),
                                     1,
                                     replace=False,
                                     p=probabilities)

        return reachable_edges[to_return[0]][1]

    def increase_trail(self):
        for ant in range(self.number_of_ants):
            ant_s = self.ants[ant]
            ant_s_distance = self.ants_distances[ant]

            for i in range(self.tsp.dimension - 1):
                self.trails[(ant_s[i], ant_s[i+1])] += self.Q / ant_s_distance

            self.trails[(ant_s[-1], ant_s[0])] += self.Q / ant_s_distance

    def decrease_trails(self):
        self.trails = {k: max(v  - self.rho, self.initial_theta) for k, v in self.trails.items()}

    def ant_colony(self, number_of_ants: int = 42, alpha: float = 2, beta: float = 2,
                   rho: float = .2, generations_number: int = 20, initial_theta: float = 0.01,
                   q: float = 500):
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.generations_number = generations_number
        self.number_of_ants = number_of_ants
        self.initial_theta = initial_theta
        self.Q = q
        self.init_trails()

        best_solution = []
        distance_best_solution = inf

        for gen in range(self.generations_number):
            self.init_ants()
            for ant in range(number_of_ants):
                for _ in range(self.tsp.dimension - 1):
                    new_node = self.choose_next_node(ant)
                    self.ants[ant].append(new_node)

                self.ants_distances[ant] = self.tsp.distance(self.ants[ant]) #Just to optimize calculations
                # select best ant
                if self.ants_distances[ant] < distance_best_solution:
                    distance_best_solution = self.ants_distances[ant]
                    best_solution = self.ants[ant]

            self.increase_trail()
            self.decrease_trails()



        #print(self.trails)
        return best_solution, distance_best_solution

##### **Solución para la busqueda por colonia de hormigas**
La mejor conseguida es de 1313 que se acerca mucho a la mejor 1270.

In [ ]:
rs = AntColony(tsp)

solution, distance = rs.ant_colony()
print("La mejor solución es {} con una distancia de {}".format(solution, distance))

La mejor solución es [0, 1, 6, 4, 3, 27, 2, 28, 29, 30, 38, 22, 39, 21, 40, 24, 41, 23, 9, 8, 10, 25, 11, 12, 18, 26, 5, 13, 19, 14, 16, 15, 37, 7, 17, 31, 36, 35, 33, 20, 34, 32] con una distancia de 1313


#### **Anexo**
Incluyo en esta sección la bibliografía consultada aunque de la mayoría no dispongo de link, puesto que me he ido bajando diferentes trabajos y al final he seleccionado los que me han parecido más interesantes sobre todo para el algoritmo de la colonia de hormigas.

- [Wikipedia](https://en.wikipedia.org/wiki/Ant_colony_optimization_algorithms)
- [Artículo de Fernando Sancho Caparrini](http://www.cs.us.es/~fsancho/?e=71) 
- [Note on the Parameter of Evaporation in the AntColony Optimization Algorithm](http://www.m-hikari.com/imf-2011/33-36-2011/kumarpIMF33-36-2011.pdf)
- Memoria de un TFM de la Universidad Complutense.
- Algoritmo ACO aplicado al TSP: Resumen de una experiencia práctica departamento de informática de la USM.